In [1]:
# Install requirements prior to running this notebook (instructions in read.me)

# Run Imports
import cv2
import mediapipe as mp
import numpy as np
import time

In [2]:
# Intialise MediaPipe Pose setup
mp_pose = mp.solutions.pose   # mp.solutions.pose is a module that handles human pose detection. mp_pose is just shorthand
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)   # Pose() creates a pose object that does the detection. Can pass parameters to Pose() to aid detection
mp_draw = mp.solutions.drawing_utils   # drawing_utils is helper module that can draw landmarks and connections on images (visualisation tool).

I0000 00:00:1764717901.910966  193643 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1764717901.926648  193712 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.2), renderer: Mesa Intel(R) UHD Graphics (TGL GT1)


In [3]:
# Functions

def calculate_angle(a, b, c):
    """ 
    Calculates the angle at point b given 3 points a, b, c
    AB and BC are the two lines. B is the point at which the angle is computed
    This is convinent for us given that it returns landmarks as points
    View ReadME to see what points are relevant and where they are (11-16) 
    """

    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ba = a-b
    bc = c-b

    # A.B = |A|x|B| cos(theta)
    # theta = arcos( (A.B)/(|A|*|B|) )
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))   
    angle = np.arccos(cosine_angle)
    
    return np.degrees(angle)


def landmark_to_pixel(landmark, image):
    """
    Convert normalized MediaPipe landmark to pixel coordinates
    """
    h, w, _ = image.shape
    return int(landmark.x * w), int(landmark.y * h)

In [ ]:
# start webcam
cap = cv2.VideoCapture(0)  # (0 = default webcam. This can be 0, 1, 2, etc. depending on the number of cameras)
# cap is now an object that will grab frames one by one

last_print_time = 0
print_interval = 0.1  # seconds

while True:   # inf loop to keep reading

    ret, frame = cap.read()  # read() captures a frame from the webcam
    # ret -> boolean. True if a frame was sucessfully captured
    # frame is the image aarray (height x width x 3 colour channels)
    if not ret:  
        break  # if ret is False, exit loop (smth went wrong)


    # Convert BGR (OpenCV) to RGB (MediaPipe)
     # OpenCV reads immages in BGR colours but Media Pipe expects RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    

    # Process frame with MediaPipe Pose
    # process() passes the RGB Frame to the MediaPipe Pose Detector
    # reults contains all detected pose landmarks and tracking information
    results = pose.process(rgb_frame)

    # results.pose_landmarks() holds all the keypoints. (elbows, shoulders, hips, etc.) 
    # if a person is detected
    image = frame
    # Draw skeleton if pose detected (if a person is detected)
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        confidence_threshold = 0.5

        # Get points for left arm
        left_hip      = [landmarks[23].x, landmarks[23].y]
        left_shoulder = [landmarks[11].x, landmarks[11].y]
        left_elbow    = [landmarks[13].x, landmarks[13].y]
        left_wrist    = [landmarks[15].x, landmarks[15].y]
        left_thumb    = [landmarks[21].x, landmarks[21].y]  

        # Get points for right arm
        right_hip      = [landmarks[24].x, landmarks[24].y]
        right_shoulder = [landmarks[12].x, landmarks[12].y]
        right_elbow    = [landmarks[14].x, landmarks[14].y]
        right_wrist    = [landmarks[16].x, landmarks[16].y]
        right_thumb    = [landmarks[22].x, landmarks[22].y]

        # Angles

        # Left
        if (landmarks[11].visibility > confidence_threshold and
            landmarks[13].visibility > confidence_threshold and
            landmarks[15].visibility > confidence_threshold):
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        else:
            left_elbow_angle = 0

        if (landmarks[11].visibility > confidence_threshold and
            landmarks[13].visibility > confidence_threshold and
            landmarks[23].visibility > confidence_threshold):
            left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
        else:
            left_shoulder_angle = 0.0

        if (landmarks[21].visibility > confidence_threshold and
            landmarks[13].visibility > confidence_threshold and
            landmarks[15].visibility > confidence_threshold):
            left_wrist_angle = calculate_angle(left_elbow, left_wrist, left_thumb)
        else:
            left_wrist_angle = 0.0



        # Right
        if (landmarks[12].visibility > confidence_threshold and
            landmarks[14].visibility > confidence_threshold and
            landmarks[16].visibility > confidence_threshold):
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        else:
            right_elbow_angle = 0.0

        if (landmarks[12].visibility > confidence_threshold and
            landmarks[14].visibility > confidence_threshold and
            landmarks[24].visibility > confidence_threshold):
            right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
        else:
            right_shoulder_angle = 0.0

        if (landmarks[22].visibility > confidence_threshold and
            landmarks[14].visibility > confidence_threshold and
            landmarks[16].visibility > confidence_threshold):
            right_wrist_angle = calculate_angle(right_elbow, right_wrist, right_thumb)
        else:
            right_wrist_angle = 0.0

        # Printing
        current_time = time.time()
        if current_time - last_print_time > print_interval:
            print(f"Left -> elbow: {left_elbow_angle:.1f} deg, shoulder: {left_shoulder_angle:.1f}  |  Right -> elbow: {right_elbow_angle:.1f} deg, shoulder: {right_shoulder_angle:.1f} ")


        # Drawings/Image

        mp_draw.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        # draw_landmarks draws small dots for keypoints and connections (lines connecting joints according to mp_pose.CONNECTIONS)
        # frame is passed/drawn on instead of rgb_frame bc we draw on frame and then display it later using cv2.imshow
    

        # converting to pixel coordinates
        right_elbow_px = landmark_to_pixel(landmarks[14], image)
        right_shoulder_px = landmark_to_pixel(landmarks[12], image)
        right_wrist_px = landmark_to_pixel(landmarks[16], image)

        left_elbow_px  = landmark_to_pixel(landmarks[13], image)
        left_shoulder_px  = landmark_to_pixel(landmarks[11], image)
        left_wrist_px = landmark_to_pixel(landmarks[15], image)

        
        cv2.putText(image, str(int(right_elbow_angle)), right_elbow_px, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        cv2.putText(image, str(int(right_shoulder_angle)), right_shoulder_px, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        cv2.putText(image, str(int(right_elbow_angle)), right_wrist_px, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

        cv2.putText(image, str(int(left_elbow_angle)), left_elbow_px, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)
        cv2.putText(image, str(int(left_shoulder_angle)), left_shoulder_px, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)
        cv2.putText(image, str(int(left_wrist_angle)), left_wrist_px, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)



    # Display: imshow opens a window named "Pose Tracking" that shows the current frame with the skelton overlay
    cv2.imshow("Pose Tracking", image)


    # code to check if 'q' is pressed -> 'q' to quit/break loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



cap.release()  # frees webcam so other programs can use it
cv2.destroyAllWindows()  # closes all openCV windows

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1764717902.029195  193694 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1764717902.045655  193702 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/viraj/Documents/GitHub/me6705_final_project/venv/lib/python3.12/site-packages/cv2/qt/plugins"
/home/viraj/Documents/GitHub/me6705_final_project/venv/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Left -> elbow: 0.0 deg, shoulder: 0.0  |  Right -> elbow: 0.0 deg, shoulder: 0.0 
Left -> elbow: 0.0 deg, shoulder: 0.0  |  Right -> elbow: 0.0 deg, shoulder: 0.0 
Left -> elbow: 0.0 deg, shoulder: 0.0  |  Right -> elbow: 0.0 deg, shoulder: 0.0 
Left -> elbow: 0.0 deg, shoulder: 0.0  |  Right -> elbow: 0.0 deg, shoulder: 0.0 
Left -> elbow: 0.0 deg, shoulder: 0.0  |  Right -> elbow: 0.0 deg, shoulder: 0.0 
Left -> elbow: 0.0 deg, shoulder: 0.0  |  Right -> elbow: 0.0 deg, shoulder: 0.0 
Left -> elbow: 0.0 deg, shoulder: 0.0  |  Right -> elbow: 0.0 deg, shoulder: 0.0 
Left -> elbow: 0.0 deg, shoulder: 0.0  |  Right -> elbow: 0.0 deg, shoulder: 0.0 
Left -> elbow: 0.0 deg, shoulder: 0.0  |  Right -> elbow: 0.0 deg, shoulder: 0.0 
Left -> elbow: 0.0 deg, shoulder: 0.0  |  Right -> elbow: 0.0 deg, shoulder: 0.0 
Left -> elbow: 0.0 deg, shoulder: 0.0  |  Right -> elbow: 0.0 deg, shoulder: 0.0 
Left -> elbow: 0.0 deg, shoulder: 0.0  |  Right -> elbow: 0.0 deg, shoulder: 0.0 
Left -> elbow: 0